In [1]:
library(stepfunctions)

stepfunctions_execution_role="dummy-role"

Set up 2 basic states: Pass and Wait

In [2]:
start_pass_state = Pass$new(
  state_id="MyPassState"
)

wait_state = Wait$new(
  state_id="Wait for 3 seconds",
  seconds=3
)

Step up Lambda with Retry and Catch

In [3]:
lambda_state = LambdaStep$new(
  state_id="Convert HelloWorld to Base64",
  parameters=list(
    "FunctionName"="MyLambda", #replace with the name of your function
    "Payload"=list(
      "input"="HelloWorld")
  )
)

lambda_state$add_retry(Retry$new(
  error_equals=list("States.TaskFailed"),
  interval_seconds=15,
  max_attempts=2,
  backoff_rate=4.0
))

lambda_state$add_catch(Catch$new(
  error_equals=list("States.TaskFailed"),
  next_step=Fail$new("LambdaTaskFailed")
))

Set up Workflow

In [4]:
workflow_definition=Chain$new(list(start_pass_state, wait_state, lambda_state))

workflow = Workflow$new(
    name="MyWorkflow_v1234",
    definition=workflow_definition,
    role=stepfunctions_execution_role
)

In [5]:
workflow$render_graph()